# Tutorial 2: Calculating travel time matrices with `r5py` and estimating GHG emission factors

```{admonition} Credits:

This tutorial was written by Henrikki Tenkanen, Christoph Fink & Willem Klumpenhouwer (i.e. `r5py` developer team).
You can read the [full documentation of `r5py`](https://r5py.readthedocs.io/en/latest/user-guide/user-manual/advanced-use.html#use-a-custom-installation-of-r5) which includes much more information and detailed user manual in case you are interested in using the library for research purposes.

```

**Lesson objectives**

This tutorial focuses on utilising  `r5py` library to find shortest paths along the given street network based on travel times or distance by car for multiple origin destination pairs/zones simultaneously. First, we will learn how to calculate travel times and distances from a single origin-destination pairs in the road network graph. Then we will explore the locozmier data to build the travel time matrices, detailed itenaries and CO2 emission matrices.  

## Introduction to `r5py`

In this tutorial, we will learn how to calculate travel times and detailed public transport travel itineraries with `r5py` between H3 hexgaons spread around the city center area of Helsinki, Finland. **R5py** is a Python library for routing and calculating travel time matrices on multimodal transport networks (walk, bike, public transport, and car).
It provides a simple and friendly interface to R<sup>5</sup> (*the Rapid Realistic Routing on Real-world and Reimagined networks*) which is a [routing engine](https://github.com/conveyal/r5) developed by [Conveyal](https://conveyal.com/). `R5py` is designed to interact with [GeoPandas](https://geopandas.org) GeoDataFrames, and it is inspired by [r5r](https://ipeagit.github.io/r5r) which is a similar wrapper developed for R. `R5py` exposes some of R5’s functionality via its [Python API](reference.html), in a syntax similar to r5r’s. At the time of this writing, only the computation of travel time matrices has been fully implemented. Over time, `r5py` will be expanded to incorporate other functionalities from R5.
When calculating travel times with `r5py`, you typically need a couple of datasets: 

- **A road network dataset from OpenStreetMap** (OSM) in Protocolbuffer Binary (`.pbf`) -format: 
  - This data is used for finding the fastest routes and calculating the travel times based on walking, cycling and driving. In addition, this data is used for walking/cycling legs between stops when routing with transit. 
  - *Hint*: Sometimes you might need modify the OSM data beforehand, e.g. by cropping the data or adding special costs for travelling (e.g. for considering slope when cycling/walking). When doing this, you should follow the instructions at [Conveyal website](https://docs.conveyal.com/prepare-inputs#preparing-the-osm-data). For adding customized costs for pedestrian and cycling analyses, see [this repository](https://github.com/RSGInc/ladot_analysis_dataprep).

- **A transit schedule dataset** in General Transit Feed Specification (GTFS.zip) -format (optional):
   - This data contains all the necessary information for calculating travel times based on public transport, such as stops, routes, trips and the schedules when the vehicles are passing a specific stop. You can read about [GTFS standard from here](https://developers.google.com/transit/gtfs/reference).
   - *Hint*: `r5py` can also combine multiple GTFS files, as sometimes you might have different GTFS feeds representing e.g. the bus and metro connections. 


### Where to get these datasets?

Here are a few places from where you can download the datasets for creating the routable network:

- **OpenStreetMap data in PBF-format**:

  - [pyrosm](https://pyrosm.readthedocs.io/en/latest/basics.html#protobuf-file-what-is-it-and-how-to-get-one)  -library. Allows downloading data directly from Python (based on GeoFabrik and BBBike).
  - [pydriosm](https://pydriosm.readthedocs.io/en/latest/quick-start.html#download-data) -library. Allows downloading data directly from Python (based on GeoFabrik and BBBike).
  - [GeoFabrik](http://download.geofabrik.de/) -website. Has data extracts for many pre-defined areas (countries, regions, etc).
  - [BBBike](https://download.bbbike.org/osm/bbbike/) -website. Has data extracts readily available for many cities across the world. Also supports downloading data by [specifying your own area or interest](https://extract.bbbike.org/).
  - [Protomaps](https://protomaps.com/downloads/osm) -website. Allows to download the data with custom extent by specifying your own area of interest.


- **GTFS data**:  
  - [Transitfeeds](https://transitfeeds.com/) -website. Easy to navigate and find GTFS data for different countries and cities. Includes current and historical GTFS data. Notice: The site will be depracated in the future.  
  - [Mobility Database](https://database.mobilitydata.org) -website. Will eventually replace TransitFeeds -website. 
  - [Transitland](https://www.transit.land/operators) -website. Find data based on country, operator or feed name. Includes current and historical GTFS data.
    
 

In [472]:
import sys
import os

if "MEM_LIMIT" in os.environ:  # binder/kubernetes!
    max_memory = int(os.environ["MEM_LIMIT"]) / 2
    sys.argv.extend(["--max-memory", f"{max_memory}"])

In [473]:
import sys
sys.argv.append([
    "--r5-classpath", 
    "https://github.com/DigitalGeographyLab/r5/releases/download/v6.9-post16-g1054c1e-20230619/r5-v6.9-post16-g1054c1e-20230619-all.jar"
])

import pandas as pd
import geopandas as gpd
import osmnx as ox
import r5py
import r5py.sampledata.helsinki


### Load transport network using `r5py`

Virtually all operations of `r5py` require a transport network. In this example, we use data from Helsinki metropolitan area, which you can easily obtain from the `r5py.sampledata.helsinki` library. The files will be downloaded automatically to a temporary folder on your computer when you call the variables `*.osm_pbf` and `*.gtfs`:

In [474]:
# Download OSM data
r5py.sampledata.helsinki.osm_pbf

SampleDataSet('C:/Users/deys1/AppData/Local/r5py/sampledata/kantakaupunki.osm.pbf')

In [475]:
# Download GTFS data
r5py.sampledata.helsinki.gtfs

SampleDataSet('C:/Users/deys1/AppData/Local/r5py/sampledata/helsinki_gtfs.zip')

To import the street and public transport networks, instantiate an `r5py.TransportNetwork` with the file paths to the OSM extract and the GTFS files:

In [476]:
from r5py import TransportNetwork

# Get the filepaths to sample data (OSM and GTFS)
helsinki_osm =   "data/Helsinki_larger_region.osm.pbf" #r5py.sampledata.helsinki.osm_pbf #
helsinki_gtfs = r5py.sampledata.helsinki.gtfs

transport_network = TransportNetwork(
    # OSM data
    helsinki_osm,
    
    # A list of GTFS file(s)
    [
        helsinki_gtfs
    ]
)

At this stage, `r5py` has created the routable transport network and it is stored in the `transport_network` variable. We can now start using this network for doing the travel time calculations. 

`````{admonition} Info regarding "An illegal reflective access operation has occurred"
:class: tip
If you receive a Warning when running the cell above (*"WARNING: An illegal reflective access operation has occurred"*), it is due to using an older version of OpenJDK library (version 11). `r5r` only supports the version 11 of OpenJDK at this stage which is the reason why we also use it here. If you plan to use only `r5py` and want to get rid of the warning, you can update the OpenJDK to it's latest version with mamba:

```
$ conda activate geo

$ mamba install -c conda-forge openjdk=20
```

`````



### Load and prepare the origin and destination data (Locomizer)

Let's start by downloading a sample dataset into a geopandas `GeoDataFrame` that we can use as our destination locations. To make testing the library easier, we have prepared....

#### Recalling the single origin-destination locations from Tutorial 1

We will first define the single origin and destination `GeoDataFrame` for theis tutorial. Let's recall the origin and destination locations in our earlier tutorial and use those as our **origin** and **destination** locations here for demonstration purposes. 

In [477]:
from shapely.geometry import Point, Polygon
import osmnx as ox 

# Origin
orig_y, orig_x = 60.16874416, 24.95721918 
# Destination
dest_y, dest_x =   60.1622494, 24.9082137 

#converting into points geometry
origin_Point = Point(orig_x, orig_y)
origin_Point_df = gpd.GeoDataFrame({"geometry": [origin_Point], "name": "Origin", "id": [0]}, index=[0], crs="epsg:4326")
destination_Point = Point(dest_x, dest_y)
destination_Point_df = gpd.GeoDataFrame({"geometry": [destination_Point], "name": "Destination", "id": [1]}, index=[0], crs="epsg:4326")

##origin_Point_df.head()
##origin_Point_df.explore(color="red", marker_kwds={"radius": 10})
#destination_Point_df.head()

Now, we proceed with all the unique OD hexagon centroids in the locomizer data, which can be read as the `loco_u_df`. We have to convert the pandas `DataFrame` into a `GeoDataFrame` as follows:

In [478]:
loco_u_df= pd.read_csv("data/Locomizer_data/Loco_unique_Hexes_LatLon_Res9.csv",index_col=0)

# Create geometry objects from WKT strings
loco_u_df['geometry'] = gpd.GeoSeries.from_wkt(loco_u_df['geometry'])
loco_u_gdf = gpd.GeoDataFrame(loco_u_df, crs="epsg:4326")

We can visualise the origin, destination and the locomizer hexgons together.

In [479]:
#m = join.explore("travel_time", cmap="Greens", max_zoom=13)
m = loco_u_gdf.explore(color="blue")
m = origin_Point_df.explore(m=m,color="red", marker_kwds={"radius": 12})
m = destination_Point_df.explore(m=m,color="black", marker_kwds={"radius": 12})
m

The `origin_Point_df` and `destination_Point_df` GeoDataFrame contains a few columns, namely `id`, and `geometry`. The `id` column with unique values and `geometry` columns are required for `r5py` to work. If your input dataset does not have an `id` column with unique values, `r5py` will throw an error. 

To get a better sense of the data, let's create a map that shows the locations of the polygons and visualise the number of people living in each cell:

### Compute travel time matrix from one to one location

A travel time matrix is a dataset detailing the travel costs (e.g., time) between given locations (origins and destinations) in a study area. To compute a travel time matrix with `r5py` based on public transportation, we first need to initialize an `r5py.TravelTimeMatrixComputer` -object. As inputs, we pass following arguments for the `TravelTimeMatrixComputer`:
- `transport_network`, which we created in the previous step representing the routable transport network. 
- `origins`, which is a GeoDataFrame with one location that we created earlier (however, you can also use multiple locations as origins).
- `destinations`, which is a GeoDataFrame representing the destinations (in our case, the `points` GeoDataFrame). 
- `departure`, which should be Python's `datetime` -object (in our case standing for "22nd of February 2022 at 08:30") to tell `r5py` that the schedules of this specific time and day should be used for doing the calculations. 
   - *Note*: By default, `r5py` summarizes and calculates a median travel time from all possible connections within 10 minutes from given depature time (with 1 minute frequency). It is possible to adjust this time window using `departure_time_window` -parameter ([see details here]((https://r5py.readthedocs.io/en/stable/reference/reference.html#r5py.RegionalTask))). For robust spatial accessibility assessment (e.g. in scientific works), we recommend to use 60 minutes `departure_time_window`. 
- `transport_modes`, which determines the travel modes that will be used in the calculations. These can be passed using the options from the `r5py.TransportMode` -class. 
  - *Hint*: To see all available options, run `help(r5py.TransportMode)`.  

```{note}
In addition to these ones, the constructor also accepts many other parameters [listed here](https://r5py.readthedocs.io/en/stable/reference/reference.html#r5py.RegionalTask), such as walking and cycling speed, maximum trip duration, maximum number of transit connections used during the trip, etc. 
```


Now, we will first create a `travel_time_matrix_computer` instance as described above:

In [480]:
import datetime
from r5py import TravelTimeMatrixComputer, TransportMode

# Initialize the tool
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=origin_Point_df,
    destinations= destination_Point_df,
    departure=datetime.datetime(2022,2,22,8,30),
    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK]
)

In [481]:
# To see all available transport modes, uncomment following
# help(TransportMode)

Running this initializes the `TravelTimeMatrixComputer`, but any calculations were not done yet.
To actually run the computations, we need to call `.compute_travel_times()` on the instance, which will calculate the travel times between all points:

In [482]:
travel_time_matrix = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix.head()

,from_id,to_id,travel_time
0,0,1,27


As a result, this returns a `pandas.DataFrame` which we stored in the `travel_time_matrix` -variable. The values in the `travel_time` column are travel times in minutes between the points identified by `from_id` and `to_id`. As you can see, the `id` value in the `from_id` column is the same for all rows because we only used one origin location as input. To get a better sense of the results, let's create a travel time map based on the locomizer origin and destination data.  

### Compute travel time matrix from many to many locations

Running the calculations between all points in our sample dataset can be done in a similar manner as calculating the travel times from one origin to all destinations. 
Since, calculating these kind of all-to-all travel time matrices is quite typical when doing accessibility analyses, it is actually possible to calculate a cross-product between all points just by using the `origins` parameter (i.e. without needing to specify a separate set for destinations). `r5py` will use the same points as destinations and produce a full set of origins and destinations. We could use the `loco_u_gdf` directly.  

But this will be slow since there are `3209` unique hexes (within `loco_u_df`), leading to `10294472` possible OD pairs. On the other hand we can use the following data to calculate travel time matrix for the present tutorial.

In [483]:
loco_Origin_Hex_LatLon= pd.read_csv(r"data/Locomizer_data/loco_Origin_Hex_LatLon_9Res.csv",index_col=0)#
loco_Destination_Hex_LatLon = pd.read_csv(r"data/Locomizer_data/loco_Destination_Hex_LatLon_9Res.csv",index_col=0)#.drop_duplicates(inplace=True)

In [484]:
import h3
loco_Origin = loco_Origin_Hex_LatLon.drop_duplicates(subset=["id"])
loco_Origin_gdf = gpd.GeoDataFrame(loco_Origin, geometry=loco_Origin.apply(lambda x: Point(x.lon,x.lat),axis=1),crs="epsg:4326")
loco_Destination = loco_Destination_Hex_LatLon.drop_duplicates(subset=["id"])
loco_Destination_gdf = gpd.GeoDataFrame(loco_Destination, geometry=loco_Destination.apply(lambda x: Point(x.lon,x.lat),axis=1),crs="epsg:4326")

In [485]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins = loco_Origin_gdf,
    destinations= loco_Destination_gdf,
    departure=datetime.datetime(2022,2,22,8,30),
    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK]
)
travel_time_matrix_all = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix_all.head()

,from_id,to_id,travel_time
0,89089961267ffff,890899695b3ffff,76.0
1,89089961267ffff,8908996800bffff,64.0
2,89089961267ffff,8908996800fffff,63.0
3,89089961267ffff,8908996801bffff,70.0
4,89089961267ffff,89089968063ffff,61.0


In [486]:
#len(loco_Origin)
#len(loco_Destination)
len(travel_time_matrix_all)

4719750

As we can see from the outputs above, now we have calculated travel times from each origin hexgaon (total 1798 unique origins) to all destinations (total 2625 destination) hexagon centroids in the study area. Hence, the resulting DataFrame has almost rows (1798x2625=4719750). Based on these results, we can for example calculate the median travel time to or from a certain point, which gives a good estimate of the overall accessibility of the location in relation to other points:

In [487]:
median_times = travel_time_matrix_all.groupby("from_id")["travel_time"].median()
median_times

from_id
89089961267ffff    85.0
89089968003ffff    72.0
8908996800fffff    72.0
8908996801bffff    75.0
89089968063ffff    64.0
                   ... 
891126d76d7ffff    51.0
891126d76dbffff    50.0
891126dad87ffff    73.0
891126dada3ffff    78.0
891126dadbbffff    74.0
Name: travel_time, Length: 1798, dtype: float64

To estimate, how long does it take in general to travel between locations in our study area (i.e. what is the baseline accessibility in the area), we can calculate the mean (or median) of the median travel times showing that it is approximately 22 minutes:

In [488]:
median_times.mean()

61.53118040089087

Now we have to attach the travel times attached to each destination hexagon boundary as a geometry object, such that we can easily visualize them on a map:

In [489]:
overall_travel_time = loco_Destination_gdf.merge(median_times.reset_index(), left_on="id", right_on="from_id")
overall_travel_time["geometry"] = overall_travel_time.id.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x, geo_json=True)))
overall_travel_time.head()

,index,id,lat,lon,geometry,from_id,travel_time
0,2,8908996800fffff,60.190463,24.730225,"POLYGON ((24.72778 60.18945, 24.73046 60.18876...",8908996800fffff,72.0
1,3,8908996801bffff,60.187418,24.722887,"POLYGON ((24.72044 60.18640, 24.72312 60.18572...",8908996801bffff,75.0
2,4,89089968063ffff,60.188410,24.738274,"POLYGON ((24.73583 60.18740, 24.73851 60.18671...",89089968063ffff,64.0
3,11,89089968003ffff,60.190132,24.725095,"POLYGON ((24.72265 60.18912, 24.72533 60.18843...",89089968003ffff,72.0
4,18,8908996806bffff,60.186026,24.741193,"POLYGON ((24.73875 60.18501, 24.74143 60.18433...",8908996806bffff,65.0


Naturally, we can also visualize these values on a map:

In [490]:
overall_travel_time.explore("travel_time", cmap="Reds", scheme="natural_breaks", k=5,tiles="OpenStreetMap")#tiles="OpenStreetMap")

In the our study area, there seems to be a bit poorer travel time in the outer part of the city centre and on the edges of the region (i.e. we witness a classic edge-effect here). 

## Advanced usage

### Compute travel times with a detailed information about the routing results

Now that we are interested in more detailed routing results, it is possible to use `DetailedItinerariesComputer`. This will provide not only the same information as in the previous examples but also brings much more detailed information about the routings. When using this functionality, `r5py` produces information about the used routes for each origin-destination pair (with possibly multiple alternative routes), as well as individual trip segments and information about the used modes, public transport route-id information (e.g. bus-line number), distanes, waiting times and the actual geometry used. 

```{important}

Computing detailed itineraries is significantly more time-consuming than calculating simple travel times. As such, think twice whether you actually need the detailed information output from this function, and how you might be able to limit the number of origins and destinations you need to compute.

```

In [ ]:
from r5py import DetailedItinerariesComputer

helsinki_osm =   r5py.sampledata.helsinki.osm_pbf #
helsinki_gtfs = r5py.sampledata.helsinki.gtfs

transport_network = TransportNetwork(
    # OSM data
    helsinki_osm,
    # A list of GTFS file(s)
    [
        helsinki_gtfs
    ]
)


detailed_itineraries_computer = DetailedItinerariesComputer(
    transport_network,
    origins=origin_Point_df,
    destinations=destination_Point_df,
    departure=datetime.datetime(2022,2,22,8,30),
    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK],
    
    # With following attempts to snap all origin and destination points to the transport network before routing
    snap_to_network=True,
)
travel_details = detailed_itineraries_computer.compute_travel_details()
##travel_details_gpd = gpd.GeoDataFrame(travel_details).to_crs(3387) ## Helsinki Projected coordinate system - EPSG:3387 KKJ / Finland (zone 5)  
travel_details_gpd = gpd.GeoDataFrame(travel_details).to_crs(3067) ## Helsinki Projected coordinate system - EPSG:3387 KKJ / Finland (zone 5)  

travel_details_gpd.head()

As you can see, the result contains much more information than earlier, see the following table for explanations:

| Column             | Description                                                                                             | Data type          |
| ------------------ | --------------------------------------------------------------------------------------------------------| -------------------|
| **from_id**        | the origin of the trip this segment belongs to                                                          | any, user defined  |
| **to_id**          | the destination of the trip this segment belongs to                                                     | any, user defined  |
| **option**         | sequential number for different trip options found                                                      | int                |
| **segment**        | sequential number for segments of the current trip options                                              | int                |
| **transport_mode** | the transport mode used on the current segment                                                          | r5py.TransportMode |
| **departure_time** | the transit departure date and time used for current segment                                            | datetime.datetime  |
| **distance**       | the travel distance in metres for the current segment                                                   | float              |
| **travel_time**    | The travel time for the current segment                                                                 | datetime.timedelta |
| **wait_time**      | The wait time between connections when using public transport                                           | datetime.timedelta |
| **route**          | The route number or id for public transport route used on a segment                                     | str                |
| **geometry**       | The path travelled on a current segment (with transit, stops connected with straight lines by default)  | shapely.LineString |

### Visualize the routes on a map

In the following, we will make a nice interactive visualization out of the results, that shows the fastest routes and the mode of transport between the given origin-destination pairs (with multiple alternative trips/routes):

In [ ]:
import folium 
import folium.plugins

# Convert travel mode to string (from r5py.TransportMode object)
travel_details["mode"] = travel_details["transport_mode"].astype(str)

# Calculate travel time in minutes (from timedelta)
travel_details["travel time (min)"] = (travel_details["travel_time"].dt.total_seconds() / 60).round(2)

# Generate text for given trip ("origin" to "destination")
travel_details["trip"] = travel_details["from_id"].astype(str) + " to " + travel_details["to_id"].astype(str)

# Choose columns for visualization
selected_cols = ["geometry", "distance", "mode", "route", "travel time (min)", "trip", "from_id", "to_id", "option", "segment"  ]

# Generate the map
m2 = travel_details[selected_cols].explore(
    tooltip=["trip", "option", "segment", "mode", "route", "travel time (min)", "distance"],
    column="mode",
    tiles="CartoDB.Positron",
    )

# Add marker for the origin
m2 = origin_Point_df.explore(m=m2, marker_type="marker", marker_kwds=dict(icon=folium.Icon(color="green", icon="train", prefix="fa", )))

# Add customized markers for destinations
destination_Point_df.apply(lambda row: (
        # Marker with destination ID number attached to the icon
        folium.Marker(
            (row["geometry"].y, row["geometry"].x),
            icon=folium.plugins.BeautifyIcon(
                icon_shape="marker",
                number=row["id"],
                border_color="#728224",
                text_color="#728224",
            )
        # Add the marker to existing map    
        ).add_to(m2)), axis=1,
)

m2



### Load and prepare the GHG emission data

Let us now import the GHG emissions per passenger-kilometer (g CO<sub>2</sub>/pkm) by transport modes Data from the file ["LCA_gCO2_per_pkm_by_transport_mode.csv"](data/LCA_gCO2_per_pkm_by_transport_mode.csv).



In [ ]:
import pandas as pd
ghg_pkm_pv = pd.read_csv("data/LCA_gCO2_per_pkm_by_transport_mode.csv",index_col=0)
ghg_pkm_pv.loc['Total_gCO2'] = ghg_pkm_pv.sum(axis=0)
ghg_pkm_pv.head()

If we know the exact distances travelled in each leg of a PT journey in a particular travel mode, we can calculate the CO2 emissions per passenger-kilometre (g CO2/km) using the above table. The DetailedItinerariesComputer provides the details of the PT trips with the travel mode. But it can be observed that the travel distances are reported as `NaN`/`None` in the output. We now have to overcome this hurdle in the following manner. 

```{admonition} Geometries of public transport routes, and distances travelled
:class: important

The default version of R⁵ is configured for performance reasons in a way that it **does not read the geometries included in GTFS data sets**.

As a consequence, the `geometry` reported by `DetailedItinerariesComputer` are straight lines in-between the stops of a public transport line, and do not reflect the actual path travelled in public transport modes.

With this in mind, *r5py* does not attempt to compute the distance of public transport segments if `SAVE_SHAPES = false`, as distances would be very crude
approximations, only. Instead it reports `NaN`/`None`.

The [Digital Geography Lab](https://www.helsinki.fi/en/researchgroups/digital-geography-lab) maintains a patched version of R⁵ in its [GitHub
repositories](https://github.com/DigitalGeographyLab/r5/releases). If you want to refrain from compiling your own R⁵ jar, but still would like to use detailed
geometries of public transport routes, follow the instructions in [Advanced use](https://r5py.readthedocs.io/en/latest/user-guide/user-manual/advanced-use.html#use-a-custom-installation-of-r5) of `r5py` documentation.
```
Hence, we have to calculate the projected distances for different travel legs of the PT journey in the following way.

In [ ]:
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
s = gpd.GeoSeries(travel_details_gpd.geometry.to_list())
travel_details_gpd["Proj_distance"] = s.length.round(2)
travel_details["Proj_distance"] = s.length.round(2)
travel_details.head(50)
unique_transit_modes = travel_details.transport_mode.unique()
unique_transit_modes
##co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Bus - ICE', 'Bus - HEV', 'Bus - BEV','Bus - BEV (two packs)', 'Bus - FCEV']].mean()
##co2_value

We need to map each unique transit modes to the relevant transit realted carbon emission factors. This is done by creating a dictionary as below:

In [ ]:
## Create a dictionary of travel mode to co2 emission factors

gross_CO2_emission_List = []
for item in unique_transit_modes:
    print(item)
    if str(item) == "TransportMode.WALK":
        temp_co2_value = 0
    elif str(item) == "TransportMode.TRAM":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Metro/urban train']].mean()
    elif str(item) == "TransportMode.SUBWAY":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Metro/urban train']].mean()
    elif str(item)  == "TransportMode.RAIL":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Metro/urban train']].mean()
    elif str(item) == "TransportMode.BUS":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Bus - ICE', 'Bus - HEV', 'Bus - BEV','Bus - BEV (two packs)', 'Bus - FCEV']].mean()
    else:
        print(str(item))
        print("Unknown Transit mode found!")
    gross_CO2_emission_List.append(temp_co2_value)
        
co2_emission_dict = {modes:co2_value for modes,co2_value in zip(unique_transit_modes, gross_CO2_emission_List)} 
co2_emission_dict

In [ ]:
travel_details['ghg_co2_emission'] = travel_details['transport_mode'].map(co2_emission_dict) * travel_details['Proj_distance']/1000
travel_details.to_csv("temp_travel_details.csv",header=True)
travel_details.round(2).head()

As a result, now we have a nice map that shows alternative routes between the origin and the given destinations in the study area. 
If you hover over the lines, you can see details about the selected routes with useful information about the travel time, distance, route id (line number) etc.
Hence, as such, if you're feeling nerdy (and happen to have Python installed to your phone 😛), you could replace your Google Maps navigator or other journey planners with `r5py`! 🤓😉

## Calculating aggregated GHG emissions due to PT trips 

### Loading the sample locomizer data

Here, we will apply the GHG emission factors for a single PT trip to the localizer trip data. Now, let us recall the following data from our earlier tutorial. This locomizer data contains all average OD trips over all Thursdays in the month of May and June for the year 2023 in the Helsinki region. 

We aim to create a square matrix where rows and columns are the hexagon IDs of the h3 OD hexagons. The matrix entities are the total GHG emissions due to PT trips according to the locomizer data.  

In [ ]:
##loco_df_raw = pd.read_csv(r"data/Locomizer_data/Locomizer_grouped_geodata_od_filtered_5.csv",index_col=0).reset_index()
loco_df_all = pd.read_csv(r"data/Locomizer_data/Locomizer_Valid_OD_Hexes_filtered.csv",index_col=0)
loco_df_all.head()

### Calculating detailed itineraries for locomizer data in r5r
We aim to create a square matrix where rows and columns are the hexagon IDs of the h3 OD hexagons. The matrix entities are the total GHG emissions due to PT trips according to the locomizer data.  

In [ ]:
## Calculate detailedItenaries (this part is calculated using r5r for faster computation) # precalculated
Loco_vHex_travel_details = pd.read_csv(r"data/Locomizer_data/Loco_Helsinki_vOD_Hex_DetIte_r5r.csv", sep=",")
Loco_vHex_travel_details.head()
len(Loco_vHex_travel_details[["from_id","to_id"]].drop_duplicates())

In [ ]:
valid_PT_OD = Loco_vHex_travel_details.set_index(["from_id","to_id"]).index.to_list()
NO_PT_OD = loco_df_all[~loco_df_all.set_index(["origin_code_level_9","destination_code_level_9"]).index.isin(valid_PT_OD)]
len(NO_PT_OD[["destination_code_level_9","origin_code_level_9"]].drop_duplicates())
#NO_PT_OD

In [ ]:
loco_valid_df = loco_df_all[loco_df_all.set_index(["origin_code_level_9","destination_code_level_9"]).index.isin(valid_PT_OD)].copy()

In [ ]:
## Create a dictionary of travel mode to co2 emission factors
unique_transit_modes = Loco_vHex_travel_details["mode"].unique()
gross_CO2_emission_List = []
for item in unique_transit_modes:
    print(item)
    if str(item) == "WALK":
        temp_co2_value = 0
    elif str(item) == "TRAM":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Metro/urban train']].mean()
    elif str(item) == "SUBWAY":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Metro/urban train']].mean()
    elif str(item)  == "RAIL":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Metro/urban train']].mean()
    elif str(item) == "BUS":
        temp_co2_value =  ghg_pkm_pv.loc['Total_gCO2',['Bus - ICE', 'Bus - HEV', 'Bus - BEV','Bus - BEV (two packs)', 'Bus - FCEV']].mean()
    else:
        print(str(item))
        print("Unknown Transit mode found!")
    gross_CO2_emission_List.append(temp_co2_value)
        
co2_emission_dict = {modes:co2_value for modes,co2_value in zip(unique_transit_modes, gross_CO2_emission_List)} 
co2_emission_dict

In [ ]:
#Loco_vHex_travel_details['Transport_mode']= Loco_vHex_travel_details["mode"].apply(lambda x: "TransportMode."+str(x))
Loco_vHex_travel_details['ghg_co2_emission'] = Loco_vHex_travel_details['mode'].map(co2_emission_dict) * Loco_vHex_travel_details['distance']/1000
Loco_vHex_travel_details.head()

In [ ]:
Loco_vHex_Agg =  Loco_vHex_travel_details.groupby(['from_id','to_id','option'])['ghg_co2_emission'].sum().reset_index()#transform('sum')#.agg.('distance')head()
##groupby options, then calculate carbon emissions
Loco_vHex_PTavgGHG = Loco_vHex_Agg.groupby(['from_id','to_id'])['ghg_co2_emission'].mean().reset_index()#head(10)
##travel_details['ghg_co2_emission'] = travel_details['transport_mode'].map(co2_emission_dict) * travel_details['Proj_distance']/1000
#Loco_Hex_travel_details_v.groupby(['from_id', 'to_id','option']).agg('total_distance')
Loco_vHex_PTavgGHG.head()
len(Loco_vHex_PTavgGHG)
#Loco_vHex_PTavgGHG.to_csv("Loco_vOD_Hex_PTavgGHG.csv",header=True)

In [ ]:
##"origin_code_level_9","destination_code_level_9"
Loco_vHex_PTavgGHG.columns
Loco_vHex_PTavgGHG.rename(columns={'from_id': "origin_code_level_9", 'to_id': "destination_code_level_9", 'ghg_co2_emission'
                                  : 'ghg_co2_emission_PT'}, inplace=True) 


### Recalling Helsinki Regional Transport Authority (HSL) survey data
Here, we recall the HSL survey on the number of trips within the residents of the Helsinki region and the survey report contains the following ratio of travel mode share in the total HSL area:

In [ ]:
Car_share_Hsl = 0.35 ## Henkilöauto
PT_share_Hsl = 0.23 ## Joukkoliikenne
Bike_share_Hsl = 0.08  ## Polkupyörä
Walk_share_Hsl = 0.33  ## Kävely
Other_share_Hsl = 0.01 ## Muu

We aim to create a square matrix where rows and columns are the hexagon IDs of the h3 OD hexagons. The matrix entities are the total GHG emissions due to car trips according to the locomizer data.  

In [ ]:
#hexagon_OD_df= pd.read_csv("data/Helsinki_6DHexagons_with_SPs_CarTravelTime_and_CO2emissions.csv",index_col=0)
PT_OD_co2_df = Loco_vHex_PTavgGHG[["origin_code_level_9","destination_code_level_9","ghg_co2_emission_PT"]].set_index(["origin_code_level_9","destination_code_level_9"])
PT_OD_co2_df.tail()

In [ ]:
PT_OD_co2_dict = PT_OD_co2_df.to_dict()
#PT_OD_co2_dict
#PT_OD_co2_dict['ghg_co2_emission_PT'][('8908996dc0bffff','891126d6adbffff')] #('8908996dc0bffff', '891126d6adbffff')

In [ ]:
## destination_code_level_9	origin_code_level_9
#loco_df_May = loco_df_[loco_df_['origin_code_level_9'] != loco_df_['destination_code_level_9']].copy()#drop_duplicates(subset=['ORIGIN_CODE_R9','DESTINATION_CODE_R9'],keep=False)#.set_index(["ORIGIN_CODE_R9","DESTINATION_CODE_R9"])
loco_valid_df.head()

In [ ]:
#loco_df_May.apply(lambda x: print(x.ORIGIN_CODE_R9,x.DESTINATION_CODE_R9),axis=1)
x = loco_valid_df.iloc[1]#["ORIGIN_CODE_R9","DESTINATION_CODE_R9"]
x.origin_code_level_9,x.destination_code_level_9
PT_OD_co2_dict['ghg_co2_emission_PT'][(x.origin_code_level_9,x.destination_code_level_9)]
loco_valid_df["Single_PT_Trip_Co2"] = loco_valid_df.apply(lambda x: PT_OD_co2_dict['ghg_co2_emission_PT'][(x.origin_code_level_9,x.destination_code_level_9)],axis=1)


In [ ]:
loco_valid_df["Agg_PT_Trip_Co2"] = PT_share_Hsl * loco_valid_df["Single_PT_Trip_Co2"] * loco_valid_df["EXTRAPOLATED_NUMBER_OF_USERS"]
loco_valid_df.head()

In [ ]:
Agg_co2_loco_PT= loco_valid_df.groupby(["destination_code_level_9",	"origin_code_level_9"]).agg('sum').reset_index()
Agg_co2_loco_PT.round(2).head()

In [ ]:
#OD_loco_May = Total_loco_df_May.pivot(index='ORIGIN_CODE_R9', columns='DESTINATION_CODE_R9', values='Agg_CarTrip_Co2')
#OD_loco_May.fillna("0").head(20)
#OD_loco_May.to_csv("TotalCo2_OD_Matrix_locomizer_May2024.csv",header=True)

In [ ]:
##loco_df_ = loco_df_all[loco_df_all.DAY == "2023-05-28"].copy()
##loco_df_May = loco_df_[loco_df_['ORIGIN_CODE_R9'] != loco_df_['DESTINATION_CODE_R9']].copy()#drop_duplicates(subset=['ORIGIN_CODE_R9','DESTINATION_CODE_R9'],keep=False)#.set_index(["ORIGIN_CODE_R9","DESTINATION_CODE_R9"])
##loco_df_May.head()
#loco_df_May.apply(lambda x: print(x.ORIGIN_CODE_R9,x.DESTINATION_CODE_R9),axis=1)
##x = loco_df_May.iloc[1]#["ORIGIN_CODE_R9","DESTINATION_CODE_R9"]
##x.ORIGIN_CODE_R9,x.DESTINATION_CODE_R9
##car_OD_co2_dict['ghg_co2_emission_Car'][(x.ORIGIN_CODE_R9,x.DESTINATION_CODE_R9)]
##loco_df_May["Single_CarTrip_Co2"] = loco_df_May.apply(lambda x: car_OD_co2_dict['ghg_co2_emission_Car'][(x.ORIGIN_CODE_R9, x.DESTINATION_CODE_R9)],axis=1)
## dict[column][key] = value

##from r5py import DetailedItinerariesComputer

##detailed_itineraries_computer = DetailedItinerariesComputer(
##    transport_network,
##    origins= hexagon_Centroid_gdf,
##    destinations= hexagon_Centroid_gdf,
##    departure=datetime.datetime(2022,2,22,8,30),
##    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK], 
    # With following attempts to snap all origin and destination points to the transport network before routing
##    snap_to_network=True,)
##All_Hex_travel_details = detailed_itineraries_computer.compute_travel_details()
##travel_details_gpd = gpd.GeoDataFrame(travel_details).to_crs(3387) ## Helsinki Projected coordinate system - EPSG:3387 KKJ / Finland (zone 5)  
##All_Hex_travel_details_gpd = gpd.GeoDataFrame(All_Hex_travel_details).to_crs(3067) ## Helsinki Projected coordinate system - EPSG:3387 KKJ / Finland (zone 5)  


## Where to go next?

In case you want to learn more, we recommend reading:
- [r5py documentation](https://r5py.readthedocs.io/) that provides much more details on how to use `r5py`.